In [0]:
%fs ls /FileStore/tables/

path name size modificationTime dbfs:/FileStore/tables/payments.csv payments.csv 57666115 1768013774000 dbfs:/FileStore/tables/riders.csv riders.csv 5594949 1768013597000 dbfs:/FileStore/tables/stations.csv stations.csv 49552 1768013601000 dbfs:/FileStore/tables/trips.csv trips.csv 440125504 1768014205000

In [0]:
# Bronze Layer Extraction
path = "/FileStore/tables/"

# 1. TRIPS
df_trips = spark.read.format("csv") \
    .option("header", "false") \
    .option("inferSchema", "true") \
    .load(f"{path}trips.csv") \
    .toDF("trip_id", "rideable_type", "start_at", "end_at", "start_station_id", "end_station_id", "rider_id")
df_trips.write.format("delta").mode("overwrite").save("/mnt/delta/bronze_trips")

# 2. RIDERS
df_riders = spark.read.format("csv") \
    .option("header", "false") \
    .option("inferSchema", "true") \
    .load(f"{path}riders.csv") \
    .toDF("rider_id", "first_name", "last_name", "address", "birthday", "account_start", "account_end", "is_member")
df_riders.write.format("delta").mode("overwrite").save("/mnt/delta/bronze_riders")

# 3. STATIONS
df_stations = spark.read.format("csv") \
    .option("header", "false") \
    .option("inferSchema", "true") \
    .load(f"{path}stations.csv") \
    .toDF("station_id", "name", "latitude", "longitude")
df_stations.write.format("delta").mode("overwrite").save("/mnt/delta/bronze_stations")

# 4. PAYMENTS
df_payments = spark.read.format("csv") \
    .option("header", "false") \
    .option("inferSchema", "true") \
    .load(f"{path}payments.csv") \
    .toDF("payment_id", "date", "amount", "rider_id")
df_payments.write.format("delta").mode("overwrite").save("/mnt/delta/bronze_payments")

In [0]:
# load step (silver layer)
spark.sql("CREATE TABLE IF NOT EXISTS staging_trips USING DELTA LOCATION '/mnt/delta/bronze_trips'")
spark.sql("CREATE TABLE IF NOT EXISTS staging_riders USING DELTA LOCATION '/mnt/delta/bronze_riders'")
spark.sql("CREATE TABLE IF NOT EXISTS staging_stations USING DELTA LOCATION '/mnt/delta/bronze_stations'")
spark.sql("CREATE TABLE IF NOT EXISTS staging_payments USING DELTA LOCATION '/mnt/delta/bronze_payments'")

Out[2]: DataFrame[]

In [0]:
# Tranformation step  (dimension tables)
# 1. Dimension: Riders (Updated account_start)
spark.sql("""
CREATE TABLE IF NOT EXISTS dim_rider
USING DELTA
AS SELECT 
    rider_id, 
    first_name, 
    last_name, 
    address, 
    birthday, 
    account_start, 
    is_member
FROM staging_riders
""")

# 2. Dimension: Stations (No changes needed)
spark.sql("""
CREATE TABLE IF NOT EXISTS dim_station
USING DELTA
AS SELECT 
    station_id, name, latitude, longitude
FROM staging_stations
""")

# 3. Dimension: Date
spark.sql("""
CREATE TABLE IF NOT EXISTS dim_date
USING DELTA
AS SELECT 
    start_at as date_id,
    date(start_at) as full_date,
    dayofweek(start_at) as day_of_week,
    month(start_at) as month,
    quarter(start_at) as quarter,
    year(start_at) as year
FROM staging_trips
UNION
SELECT 
    date as date_id,
    date(date) as full_date,
    dayofweek(date) as day_of_week,
    month(date) as month,
    quarter(date) as quarter,
    year(date) as year
FROM staging_payments
""")

Out[4]: DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:

from pyspark.sql.functions import col, unix_timestamp, year

# Load staging data
trips = spark.table("staging_trips")
riders = spark.table("staging_riders")
dates = spark.table("dim_date")
stations = spark.table("dim_station")

# 4. Fact: Trips (Explicit joins as requested by reviewer)
fact_trip = trips.alias("t") \
    .join(riders.alias("r"), "rider_id") \
    .join(dates.alias("d"), col("t.start_at") == col("d.date_id")) \
    .join(stations.alias("ss"), col("t.start_station_id") == col("ss.station_id")) \
    .join(stations.alias("es"), col("t.end_station_id") == col("es.station_id")) \
    .select(
        col("t.trip_id"),
        col("t.rider_id"),
        col("t.start_station_id"),
        col("t.end_station_id"),
        col("d.date_id"),
        ((unix_timestamp("t.end_at") - unix_timestamp("t.start_at")) / 60).alias("duration"),
        (year("t.start_at") - year("r.birthday")).alias("rider_age")
    )

fact_trip.write.format("delta").mode("overwrite").saveAsTable("fact_trip")

# 5. Fact: Payments
fact_payment = spark.table("staging_payments").alias("p") \
    .join(dates.alias("d"), col("p.date") == col("d.date_id")) \
    .select(
        col("p.payment_id"),
        col("p.rider_id"),
        col("d.date_id"),
        col("p.amount")
    )

fact_payment.write.format("delta").mode("overwrite").saveAsTable("fact_payment")

In [0]:
%sql
-- Check all tables in your current database
SHOW TABLES;

-- Verify the columns in your Fact Table
DESCRIBE TABLE fact_trip;

col_name data_type comment trip_id string rider_id int start_station_id string end_station_id string date_id timestamp duration double rider_age int # Partitioning Not partitioned

In [0]:
%sql
SELECT 
    trip_id, 
    duration, 
    rider_age 
FROM fact_trip 
LIMIT 10;

trip_id duration rider_age 0FF8A33DAE295464 18.216666666666665 35 92075A0BC37B3604 24.533333333333335 16 5EC7D0B7DD45317B 14.483333333333333 29 590A21449F700AAB 61.3 33 AD1244D9D669DEA5 11.133333333333333 37 14A780860CA70DA0 5.5 52 82E76046FB3DDB35 5.7 26 4B4C7EE379092FAA 13.116666666666667 23 97DF54420116CC2B 16.116666666666667 30 1FCB3C1212DE838E 17.65 32

In [0]:
%sql
SELECT 
    t.trip_id,
    s_start.name AS start_station,
    s_end.name AS end_station,
    d.year,
    d.month,
    t.duration
FROM fact_trip t
JOIN dim_station s_start ON t.start_station_id = s_start.station_id
JOIN dim_station s_end ON t.end_station_id = s_end.station_id
JOIN dim_date d ON t.date_id = d.date_id
LIMIT 10;

trip_id start_station end_station year month duration 0FF8A33DAE295464 Delano Ct & Roosevelt Rd Clinton St & Lake St 2021 6 18.216666666666665 92075A0BC37B3604 Shedd Aquarium Shore Dr & 55th St 2021 6 24.533333333333335 5EC7D0B7DD45317B Franklin St & Jackson Blvd Clinton St & Lake St 2021 6 14.483333333333333 590A21449F700AAB Ashland Ave & Augusta Blvd Ashland Ave & Grand Ave 2021 6 61.3 AD1244D9D669DEA5 Fairbanks Ct & Grand Ave Clinton St & Lake St 2021 6 11.133333333333333 14A780860CA70DA0 Racine Ave & Washington Blvd Clinton St & Lake St 2021 6 5.5 82E76046FB3DDB35 Damen Ave & Chicago Ave Ashland Ave & Grand Ave 2021 6 5.7 4B4C7EE379092FAA Canal St & Jackson Blvd Ashland Ave & Grand Ave 2021 6 13.116666666666667 97DF54420116CC2B Museum of Science and Industry Shore Dr & 55th St 2021 6 16.116666666666667 1FCB3C1212DE838E Michigan Ave & Lake St Ashland Ave & Grand Ave 2021 6 17.65

In [0]:
# Run this in a Python cell
staging_count = spark.table("staging_trips").count()
fact_count = spark.table("fact_trip").count()

print(f"Staging Rows: {staging_count}")
print(f"Fact Rows: {fact_count}")

if staging_count == fact_count:
    print("Verification Successful: No data lost!")
else:
    print("Warning: Row counts do not match.")

Staging Rows: 4584921
Fact Rows: 4584921
Verification Successful: No data lost!

In [0]:
%sql
-- Check all tables in your current database
SHOW TABLES;

database tableName isTemporary default dim_date false default dim_rider false default dim_station false default fact_payment false default fact_trip false default staging_payments false default staging_riders false default staging_stations false default staging_trips false